In [15]:
import pandas as pd
import datetime
import numpy as np

from math import sqrt, log
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

from sklearn import linear_model

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot

from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor

In [16]:
df = pd.read_csv('/Users/gonca/OneDrive/Ambiente de Trabalho/IronKaggle/Data_final.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [17]:
df

,Store_ID,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales,Sale_ID
0,1082,2013-06-02,0,0,0,0,0,0,613458
1,747,2014-03-17,813,1,1,0,0,10016,139432
2,623,2014-05-18,0,0,0,0,0,0,468618
3,1054,2014-04-11,595,1,0,0,0,5342,233950
4,31,2013-06-24,522,1,0,0,1,4617,94205
...,...,...,...,...,...,...,...,...,...
640835,188,2013-07-25,331,1,0,0,1,3446,452950
640836,789,2013-04-21,0,0,0,0,0,0,239543
640837,344,2014-06-26,930,1,0,0,0,6371,272832
640838,267,2014-09-24,853,1,0,0,0,6638,540013


In [18]:
# CREATE DATETIME DATA
days = []
months = []
years = []

for d in df['Date']:
    year,month,day = d.split('-')
    days.append(int(day))
    months.append(int(month))
    years.append(int(year))
df['days'] = days
df['months'] = months
df['years'] = years
df['weekday'] = [x.weekday() for x in pd.to_datetime(df['Date'],format='%Y-%m-%d')]
df.drop('Date',axis=1,inplace=True)

In [19]:
df.loc[df['weekday']>=6,'weekend_dummy']=1
df.loc[df['weekday']<6,'weekend_dummy']=0

In [20]:
df.loc[df['State_holiday']=='0','holiday_dummy']=1
df.loc[df['State_holiday']!='0','holiday_dummy']=0
#df = df.merge(pd.get_dummies(df['State_holiday']),left_index=True,right_index=True).drop('State_holiday',axis=1)
df.drop('State_holiday',axis=1,inplace=True)

In [21]:
df['weekend_dummy'].value_counts()

0.0    549702
1.0     91138
Name: weekend_dummy, dtype: int64

In [22]:
df['holiday_dummy'].value_counts()

1.0    621156
0.0     19684
Name: holiday_dummy, dtype: int64

In [23]:
df.drop('Sale_ID',axis=1,inplace=True)

In [24]:
df

,Store_ID,Nb_customers_on_day,Open,Promotion,School_holiday,Sales,days,months,years,weekday,weekend_dummy,holiday_dummy
0,1082,0,0,0,0,0,2,6,2013,6,1.0,1.0
1,747,813,1,1,0,10016,17,3,2014,0,0.0,1.0
2,623,0,0,0,0,0,18,5,2014,6,1.0,1.0
3,1054,595,1,0,0,5342,11,4,2014,4,0.0,1.0
4,31,522,1,0,1,4617,24,6,2013,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
640835,188,331,1,0,1,3446,25,7,2013,3,0.0,1.0
640836,789,0,0,0,0,0,21,4,2013,6,1.0,1.0
640837,344,930,1,0,0,6371,26,6,2014,3,0.0,1.0
640838,267,853,1,0,0,6638,24,9,2014,2,0.0,1.0


In [25]:
df.isna().sum()

Store_ID               0
Nb_customers_on_day    0
Open                   0
Promotion              0
School_holiday         0
Sales                  0
days                   0
months                 0
years                  0
weekday                0
weekend_dummy          0
holiday_dummy          0
dtype: int64

In [26]:
df = df[df['Open']==1]

In [27]:
X = df.drop('Sales',axis=1)
y = df['Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 0, test_size = 0.20)

In [23]:
X_train

,Store_ID,Nb_customers_on_day,Open,Promotion,School_holiday,days,months,years,weekday,weekend_dummy,holiday_dummy
581431,178,979,1,1,1,30,7,2014,2,0.0,1.0
594485,1067,575,1,1,0,18,2,2013,0,0.0,1.0
62190,414,592,1,0,0,7,6,2014,5,0.0,1.0
595976,233,960,1,0,0,22,7,2015,2,0.0,1.0
326108,295,274,1,0,0,12,2,2015,3,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
433479,837,506,1,0,0,29,11,2013,4,0.0,1.0
183382,310,625,1,0,0,15,2,2014,5,0.0,1.0
141997,420,377,1,1,0,16,7,2013,1,0.0,1.0
524936,545,1483,1,1,0,31,3,2014,0,0.0,1.0


In [24]:
# Linear Regression
lin_model = linear_model.LinearRegression()
lin_model.fit(X_train, y_train)

# Display training and testing results
pred = lin_model.predict(X_test)

In [25]:
r2_score(y_test, pred)

0.7361050812688494

In [127]:
sqrt(mean_squared_error(y_test, pred))

1472.8704958210772

In [128]:
mean_absolute_error(y_test, pred)

983.4133229466808

In [112]:
mean_absolute_error(y_test, y_pred)

1041.6652031011122

In [26]:
gbrt = GradientBoostingRegressor(random_state = 42, 
                                 learning_rate = 0.5, 
                                 max_depth = 6, 
                                 n_estimators = 450, 
                                 min_samples_leaf = 5)
gbrt.fit(X_train, y_train)

print("Accuracy on training set: ", gbrt.score(X_train, y_train))
print("Accuracy on test set: ", gbrt.score(X_test, y_test))


Accuracy on training set:  0.9842654127045453
Accuracy on test set:  0.9794478025205506


In [23]:
y_pred = gbrt.predict(X_test)

In [24]:
mean_absolute_error(y_test, y_pred)

313.30874989975774

In [ ]:
# get a stacking ensemble of models

def get_stacking():
    # define the base models
    level0 = list()
    
    level0.append(('random_forest', RandomForestRegressor(bootstrap = True, max_depth = 12,
                               n_estimators = 500, random_state = 0)))
    
    level0.append(('gradient_boosting', GradientBoostingRegressor(learning_rate = 0.5,
                                                             max_depth = 8,
                                                            n_estimators = 500, random_state = 0)))
    
    
    # define meta learner model
    level1 = DecisionTreeRegressor()
                  
    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
                  
    return model

def get_models():
    models = dict()
    models['random_forest'] = RandomForestRegressor(bootstrap = True, max_depth = 12,
                               n_estimators = 500, random_state = 0)
    models['gradient_boosting'] = GradientBoostingRegressor(learning_rate = 0.5,
                                                             max_depth = 8,
                                                            n_estimators = 500, random_state = 0)
    models['stacking'] = get_stacking()
    return models

def evaluate_model(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=0)
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
    return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.5f (%.5f)' % (name, np.mean(scores), np.std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

>random_forest -947.79247 (2.78169)
